<a href="https://colab.research.google.com/github/RadhavaramHarika/Harika_INFO5731_Spring2020/blob/master/Harika_In_class_exercise_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fifth in-class-exercise**

## **1. Rule-based information extraction**

Use any keywords related to data science, natural language processing, machine learning to search from google scholar, get the **titles** of 100 articles (either by web scraping or manually) about this topic, define a set of patterns to extract the research questions/problems, methods/algorithms/models, datasets, applications, or any other important information about this topic. 

In [170]:
# Write your code here
import re, numpy as nump, pandas as panda
import urllib.request
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import pandas as panda
import requests
!pip install scholarly
import scholarly
from itertools import cycle
import traceback
from lxml.html import fromstring
import string 
import nltk 
import spacy 
import numpy as nump 
import math 
from tqdm import tqdm 
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 

panda.set_option('display.max_colwidth', 200)
nlp = spacy.load("en_core_web_sm")  

#Retrieving proxies from URL to handle error 429-'Too many requests from the user' to google scholar url
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
      if i.xpath('.//td[7][contains(text(),"yes")]'):
        #Grabbing IP and corresponding PORT
        proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
        proxies.add(proxy)
    return proxies

#Retrieving article_items
def getTitles(soup):
  articles = soup.find("div", {"id": "gs_res_ccl_mid"})
  article_items = articles.find_all(class_ = "gs_r gs_or gs_scl")
  list_articles = []
  for each in article_items:
    list_articles.append(each.find(class_ = "gs_ri"))
  return list_articles

#Retrieving artictle titles and storing into a list of dicts
def getArticle_dict(articles):
  art_list = []
  for each in articles:
    art_list.append({'title':each.find('h3').get_text(),'full_title': each.find(class_ = "gs_rs").get_text()})
  return art_list
  
#Retrieving URLs for all keys
def retrieveURLs(soup):
  base_url = "https://scholar.google.com/"
  keys = soup.find("div", {"id": "gs_nml"})
  key_urls = []
  for each in keys.find_all('a'):
    if int(each.get_text())<11:
      key_urls.append(str(base_url+each.get('href')))
  return key_urls

#Retrieving URL data using beautifulsoup
def getDataFromUrl(url):
  proxies = get_proxies()
  proxy_pool = cycle(proxies)
  proxy = next(proxy_pool)
  try:
    response = requests.get(url,proxies={"http": proxy, "https": proxy}).text
    b_soup = BeautifulSoup(response,'html.parser')
    return b_soup   
  except:
    #Skipping connection errors as all above created proxies won't connect to the url and gives connection issues,
    print("Skipping. Connnection error")

#Taking POS Tags
def getPOSforText(text):
  doc = nlp(text)
  for token in doc: 
    print(token.text, "-->",token.dep_,"-->", token.pos_)
  return doc

#describing question Pattern
def questPattern(token_doc):
  pattern = [{'POS':'PROPN'}, 
             {'POS':'advmod'},
           {'LOWER': 'is'}, 
           {'LOWER':'are'},
           {'LOWER': 'was'}, 
           {'POS': 'NOUN','OP':'?'},
           {'POS':'amod','OP':'?'}]
  matcher = Matcher(nlp.vocab) 
  matcher.add("matching_1", None, pattern) 
  matches = matcher(token_doc) 
  span = token_doc[matches[0][1]:] 
  return span.text

#describing Problem Pattern
def probPattern(token_doc):
  pattern = [{'POS':'PROPN'}, 
             {'POS':'NOUN'},
           {'LOWER': 'in'}, 
           {'LOWER':'of'},
           {'LOWER': 'are'},
            {'LOWER':'were'}, 
           {'POS': 'NOUN','OP':'?'},
           {'POS':'amod','OP':'?'}]
  matcher = Matcher(nlp.vocab) 
  matcher.add("matching_1", None, pattern) 
  matches = matcher(token_doc) 
  span = token_doc[matches[0][1]:] 
  return span.text

#describing fact Pattern
def factPattern(token_doc):
  pattern = [{'POS':'PROPN'}, 
             {'POS':'NOUN'},
           {'LOWER': 'as'}, 
           {'LOWER':'a'},
           {'LOWER': 'is'},
            {'LOWER':'in'}, 
             {'LOWER':'of'},
           {'POS': 'NOUN','OP':','},
           {'POS':'VERB'}]

  matcher = Matcher(nlp.vocab) 
  matcher.add("matching_1", None, pattern) 
  matches = matcher(token_doc) 
  span = token_doc[matches[0][1]:] 
  return span.text

#Constructing url to search article in google Scholr with keywords-Pattern Recognition Machine Learning 
query = "pattern+recognition+machine+learning"
url = 'https://scholar.google.com/scholar?q=' + query + '&ie=UTF-8&oe=UTF-8&hl=en&btnG=Search'

#Retrieving all the article titles from above functions
content = getDataFromUrl(url)
print(content)
if content:
  allTitles = getTitles(content)
  print(len(allTitles))
  '''if content:
    for each in retrieveURLs(content):
      allTitles.append(getTitles(getDataFromUrl(each)))
      print(len(allTitles))'''

#Calling above function to print the patterned texts from all the articles
for each in getArticle_dict(allTitles):
  print(questPattern(getPOSforText(each['full_title'])))
  print(probPattern(getPOSforText(each['full_title'])))
  print(factPattern(getPOSforText(each['full_title'])))




ERROR: Operation cancelled by user
Skipping. Connnection error
None
This --> nsubj --> DET
is --> ROOT --> VERB
the --> det --> DET
first --> amod --> ADJ
textbook --> attr --> NOUN
on --> prep --> ADP
pattern --> compound --> NOUN
recognition --> pobj --> NOUN
to --> aux --> PART
present --> relcl --> VERB
the --> det --> DET
Bayesian --> amod --> ADJ
viewpoint --> dobj --> NOUN
. --> punct --> PUNCT
The --> det --> DET
book --> compound --> NOUN
presents --> nsubj --> VERB
approximate --> ROOT --> ADJ
inference --> compound --> NOUN
algorithms --> dobj --> NOUN
that --> nsubj --> DET
permit --> relcl --> VERB
fast --> amod --> ADJ
approximate --> amod --> ADJ
answers --> dobj --> NOUN
in --> prep --> ADP
situations --> pobj --> NOUN
where --> advmod --> ADV
exact --> amod --> ADJ
answers --> nsubj --> NOUN
are --> relcl --> VERB
not --> neg --> ADV
feasible --> acomp --> ADJ
. --> punct --> PUNCT
It --> nsubj --> PRON
uses --> ROOT --> VERB
graphical --> amod --> ADJ
models --> dobj 

IndexError: ignored

## **2. Domain-specific information extraction**

For the legal case used in the data cleaning exercise: [01-05-1 Adams v Tanner.txt](https://raw.githubusercontent.com/unt-iialab/INFO5731_Spring2020/master/In_class_exercise/01-05-1%20%20Adams%20v%20Tanner.txt), use [legalNLP](https://lexpredict-lexnlp.readthedocs.io/en/latest/modules/extract/extract.html#nlp-based-extraction-methods) to extract the following inforation from the text (if the information is not exist, just print None):

(1) acts, e.g., “section 1 of the Advancing Hope Act, 1986”

(2) amounts, e.g., “ten pounds” or “5.8 megawatts”

(3) citations, e.g., “10 U.S. 100” or “1998 S. Ct. 1”

(4) companies, e.g., “Lexpredict LLC”

(5) conditions, e.g., “subject to …” or “unless and until …”

(6) constraints, e.g., “no more than”

(7) copyright, e.g., “(C) Copyright 2000 Acme”

(8) courts, e.g., “Supreme Court of New York”

(9) CUSIP, e.g., “392690QT3”

(10) dates, e.g., “June 1, 2017” or “2018-01-01”

(11) definitions, e.g., “Term shall mean …”

(12) distances, e.g., “fifteen miles”

(13) durations, e.g., “ten years” or “thirty days”

(14) geographic and geopolitical entities, e.g., “New York” or “Norway”

(15) money and currency usages, e.g., “$5” or “10 Euro”

(16) percents and rates, e.g., “10%” or “50 bps”

(17) PII, e.g., “212-212-2121” or “999-999-9999”

(18) ratios, e.g.,” 3:1” or “four to three”

(19) regulations, e.g., “32 CFR 170”

(20) trademarks, e.g., “MyApp (TM)”

(21) URLs, e.g., “http://acme.com/”

(22) addresses, e.g., “1999 Mount Read Blvd, Rochester, NY, USA, 14615”

(23) persons, e.g., “John Doe

In [24]:
# write your code here
import nltk
import csv,os,math,unicodedata,re
from google.colab import drive,files
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
import pandas as panda
!pip install lexnlp
import lexnlp.extract.en.acts as act
import lexnlp.extract.en.citations as cite
import lexnlp.extract.en.amounts as amount
import lexnlp.extract.en.conditions as cond
import lexnlp.extract.en.constraints as const
import lexnlp.extract.en.copyright as copy
import lexnlp.extract.en.courts as court
import lexnlp.extract.en.cusip
import lexnlp.extract.en.dates
import lexnlp.extract.en.definitions
import lexnlp.extract.en.distances
import lexnlp.extract.en.durations
import lexnlp.extract.en.geoentities
import lexnlp.extract.en.money
import lexnlp.extract.en.percents
import lexnlp.extract.en.pii
import lexnlp.extract.en.ratios
import lexnlp.extract.en.trademarks
import lexnlp.extract.en.urls
import lexnlp.extract.en.regulations
import lexnlp.extract.en.entities.nltk_re
import lexnlp.extract as extract

#Opening Legal_text file from google drive
def open_textfile():
  drive.mount('/content/gdrive')
  file_list = os.listdir('/content/gdrive/My Drive/Colab Notebooks')
  os.chdir('/content/gdrive/My Drive/Colab Notebooks')
  if 'Legal_case.txt' in file_list:
    text_data = open('Legal_case.txt').read()
  clean_data = text_data.replace('\xa0','')
  return clean_data

#Splitting document into text sentences and removing non-alphanumeric characters
def getTextSentences(samp_data):
  text_sent = [re.sub(r"\W", " ", each, flags=re.I) for each in samp_data.split('\n')
                  if each]
  return text_sent

#Extracting Acts from each text sentence
def extractActs(textData):
  acts_list = []
  for each in textData:
    temp = list(act.get_act_list(each))
    if len(temp)!=0:
      acts_list.append(temp)
  return acts_list if acts_list else None

#Extracting Amounts from each text sentence
def extractAmounts(textData):
  amnt_list = []
  for each in textData:
    temp = list(amount.get_amounts(each))
    if len(temp)!=0:
     amnt_list.append(temp)
  return amnt_list if amnt_list else None

#Extracting Citations from each text sentence
def extractCitation(textData):
  cite_list = []
  for each in textData:
    temp = list(cite.get_citations(each))
    if len(temp)!=0:
     cite_list.append(temp)
  return cite_list if cite_list else None 

#Extracting Companies from each text sentence
def extractCompanies(textData):
  comp_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.entities.nltk_re.get_companies(each))
    if len(temp)!=0:
     comp_list.append(temp)
  return comp_list if comp_list else None 

#Extracting Conditions from each text sentence
def extractConditions(textData):
  cond_list = []
  for each in textData:
    temp = list(cond.get_conditions(each))
    if len(temp)!=0:
     cond_list.append(temp)
  return cond_list if cond_list else None 

#Extracting Constraints from each text sentence
def extractConstraints(textData):
  const_list = []
  for each in textData:
    temp = list(const.get_constraints(each))
    if len(temp)!=0:
     const_list.append(temp)
  return const_list if const_list else None 

#Extracting Copyrights from each text sentence
def extractCopyrights(textData):
  copy_list = []
  for each in textData:
    temp = list(copy.get_copyright(each))
    if len(temp)!=0:
     copy_list.append(temp)
  return copy_list if copy_list else None

#Extracting CUSIP from each text sentence
def extractCUSIP(textData):
  cusip_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.cusip.get_cusip(each))
    if len(temp)!=0:
     cusip_list.append(temp)
  return cusip_list if cusip_list else None

#Extracting Dates from each text sentence
def extractDates(textData):
  date_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.dates.get_dates(each))
    if len(temp)!=0:
     date_list.append(temp)
  return date_list if date_list else None

#Extracting Definitions from each text sentence
def extractDefinitions(textData):
  def_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.definitions.get_definitions(each))
    if len(temp)!=0:
     def_list.append(temp)
  return def_list if def_list else None  

#Extracting Distances from each text sentence
def extractDistances(textData):
  dist_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.distances.get_distances(each))
    if len(temp)!=0:
     dist_list.append(temp)
  return dist_list if dist_list else None

#Extracting Durations from each text sentence
def extractDurations(textData):
  dur_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.durations.get_durations(each))
    if len(temp)!=0:
     dur_list.append(temp)
  return dur_list if dur_list else None

#Extracting courts from each text sentence
def extractCourts(textData):
  file_data = panda.read_csv("https://raw.githubusercontent.com/LexPredict/lexpredict-legal-dictionary/master/en/legal/us_courts.csv",error_bad_lines=False)
  court_config = []
  courts_list = []
  for i in range(len(file_data)):
    a = str(file_data["Alias"][i]).split(";")
    alias_list = [(each,None,False,a.index(each)) for each in a]
    c = (file_data["Court ID"][i],file_data["Court Name"][i],0,alias_list)
    court_config.append(c)
  for each in textData:
    temp = {}
    for entity, alias in lexnlp.extract.en.courts.get_courts(each,court_config):
      temp["enity"] = entity
      temp["alias"] = alias
    if temp:
      courts_list.append(temp)
  return courts_list if courts_list else None

#Extracting Geographics and geopolitics from each text sentence
def extractGeoPolitics(textData):
  file_data = panda.read_csv("https://raw.githubusercontent.com/LexPredict/lexpredict-legal-dictionary/master/en/geopolitical/us_state_regulators.csv",error_bad_lines=False)
  geo_config = []
  geo_list = []
  for i in range(len(file_data)):
    a = str(file_data["Alias"][i]).split(";")
    alias_list = [(each,None,file_data["Case Sensitive"][i],a.index(each)) for each in a]
    g = (i,file_data["Term"][i],0,alias_list)
    geo_config.append(g)
  for each in textData:
    temp = {}
    for entity, alias in lexnlp.extract.en.geoentities.get_geoentities(each,geo_config):
      temp["enity"] = entity
      temp["alias"] = alias
    if temp:
      geo_list.append(temp)
  return geo_list if geo_list else None

#Extracting Money and Currency from each text sentence
def extractMoneyCurrency(textData):
  money_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.money.get_money(each))
    if len(temp)!=0:
     money_list.append(temp)
  return money_list if money_list else None

#Extracting Percent and Rates from each text sentence
def extractPercentRates(textData):
  percent_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.percents.get_percents(each))
    if len(temp)!=0:
     percent_list.append(temp)
  return percent_list if percent_list else None

#Extracting PII from each text sentence
def extractPII(textData):
  pii_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.pii.get_pii(each))
    if len(temp)!=0:
     pii_list.append(temp)
  return pii_list if pii_list else None

#Extracting Ratios from each text sentence
def extractRatios(textData):
  ratio_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.ratios.get_ratios(each))
    if len(temp)!=0:
     ratio_list.append(temp)
  return ratio_list if ratio_list else None

#Extracting Regulations from each text sentence
def extractRegulations(textData):
  reg_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.regulations.get_regulations(each))
    if len(temp)!=0:
     reg_list.append(temp)
  return reg_list if reg_list else None

#Extracting Trademarks from each text sentence
def extractTrademarks(textData):
  trad_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.trademarks.get_trademarks(each))
    if len(temp)!=0:
     trad_list.append(temp)
  return trad_list if trad_list else None

#Extracting URLs from each text sentence
def extractURLs(textData):
  url_list = []
  for each in textData:
    temp = list(lexnlp.extract.en.urls.get_urls(each))
    if len(temp)!=0:
     url_list.append(temp)
  return url_list if url_list else None

#Extracting pos_tags from each text sentence
def basicFeatExtractio(TextSent):
  stop = stopwords.words('english')
  sent = ' '.join([w for w in TextSent.split() if w not in stop])
  tokens = nltk.word_tokenize(sent)
  pos_words = nltk.pos_tag(tokens)
  return pos_words

#Extracting Address and Location from each text sentence
def extractNamesLocations(textData):
  names = []
  address = []
  for each in textData:
    name_sent = []
    address_sent = []
    tag_sent = basicFeatExtractio(each)
    for chunk in nltk.ne_chunk(tag_sent):
      if type(chunk) == nltk.tree.Tree:
          if chunk.label() == 'PERSON':
            name_sent.append(' '.join([w for w,t in chunk]))
          elif chunk.label == 'LOCATION':
            address_sent.append(' '.join([w for w,t in chunk]))
    if name_sent:
      names.append(name_sent)
    if address_sent:
      address.append(address_sent)
  return (names if names else None,address if address else None)

#Calling above functions to print results
text_data = open_textfile()
text_sent = getTextSentences(text_data)

print("\nActs :\n ")
print(extractActs(text_sent))

print("\nAmounts :\n")
print(extractAmounts(text_sent))

print("\nCitations :\n")
print(extractCitation(text_sent))

print("\nCompanies :\n")
print(extractCompanies(text_sent))

print("\nConditions :\n")
print(extractConditions(text_sent))

print("\nConstraints :\n")
print(extractConstraints(text_sent))

print("\nCopyrights :\n")
print(extractCopyrights(text_sent))

print("\nCourts :\n")
print(extractCourts(text_sent))

print("\nCUSIP :\n")
print(extractCUSIP(text_sent))

print("\nDates :\n")
print(extractDates(text_sent))

print("\nDefinitions :\n")
print(extractDefinitions(text_sent))

print("\nDistances :\n")
print(extractDistances(text_sent))

print("\nDurations :\n")
print(extractDurations(text_sent))

print("\nGeographics And Geopolitics:\n")
print(extractGeoPolitics(text_sent))

print("\nMoney And currency :\n")
print(extractMoneyCurrency(text_sent))

print("\nPercent And Rates :\n")
print(extractPercentRates(text_sent))

print("\nPII :\n")
print(extractPII(text_sent))

print("\nRatios :\n")
print(extractRatios(text_sent))

print("\nRegulstions :\n")
print(extractRegulations(text_sent))

print("\nTrademarks :\n")
print(extractTrademarks(text_sent))

print("\nURLs :\n")
print(extractURLs(text_sent))

results = extractNamesLocations(text_sent)
print("\nNames: \n",results[0])
print("\nAddress: \n",results[1])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Acts :
 
None

Amounts :

[[5.0, 740.0], [1843.0], [2.0], [1.0], [4.0], [2.0], [1821.0], [5.0], [1.0, 1840.0, 3777, 80.0, 100.0, 30, 1839.0, 741.0, 22, 1840.0, 14000, 120, 1, 1840.0, 3